<a href="https://colab.research.google.com/github/manavgurnani21/data_augmentation_tld_research/blob/main/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Adding Images to Drive Folder

In [ ]:
!pip install -q kaggle

from google.colab import files
files.upload()

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d mbornoe/lisa-traffic-light-dataset
!unzip lisa-traffic-light-dataset.zip

In [ ]:
import os

# getting current directory
os.getcwd()

all_image_paths = []

## Getting Day Sequence Paths

In [ ]:
# getting all paths for content layer
content = os.listdir('/content/')
content.sort()
content = content[:-3]
content.remove('.config')
content.remove('kaggle.json')
content.remove('lisa-traffic-light-dataset.zip')
content.remove('Annotations')
content.remove('dayTrain')
content.remove('nightTrain')
for folder in content:
  if folder == '.ipynb_checkpoints':
    content.remove('.ipynb_checkpoints')
print(content)

['daySequence1', 'daySequence2', 'nightSequence1', 'nightSequence2']


In [ ]:
for folder in content:
  print('/content/' + folder + '/' + folder + '/frames/')
  list = os.listdir('/content/' + folder + '/' + folder + '/frames/')
  for path in list:
    all_image_paths.append('/content/' + folder + '/' + folder + '/frames/' + path)

/content/daySequence1/daySequence1/frames/
/content/daySequence2/daySequence2/frames/
/content/nightSequence1/nightSequence1/frames/
/content/nightSequence2/nightSequence2/frames/


## Getting Clip Paths

In [ ]:
train_paths = ['/content/dayTrain/dayTrain/', '/content/nightTrain/nightTrain/']
for path in train_paths:
  list1 = os.listdir(path)
  if '.DS_Store' in list1:
    list1.remove('.DS_Store')
  for name in list1:
    list2 = os.listdir(path + name + '/frames/')
    for item in list2:
      all_image_paths.append(path + name + '/frames/' + item)

# Adding All Annotations

## Getting all Sequence Annotations

In [ ]:
import os

all_annotation_paths = []

In [ ]:
root_path = '/content/Annotations/Annotations/'
main = os.listdir(root_path)
main.remove('dayTrain')
main.remove('nightTrain')

for folder in main:
  list1 = os.listdir(root_path + folder)
  list1[0] = folder + list1[0]
  os.rename(root_path + folder + '/frameAnnotationsBOX.csv', root_path + folder + '/' + list1[0])
  all_annotation_paths.append(root_path + folder + '/' + list1[0])

## Getting all Clip Annotations

In [ ]:
clipPaths = [root_path + 'dayTrain/', root_path + 'nightTrain/']

for folder in clipPaths:
  list2 = os.listdir(folder)
  for name in list2:
    list3 = os.listdir(folder + name)
    list3[0] = name + list3[0]
    print(folder + name + '/' + list3[0])
    os.rename(folder + name + '/frameAnnotationsBOX.csv', folder + name + '/' + list3[0])
    all_annotation_paths.append(folder + name + '/' + list3[0])

# Sorting All Lists

In [ ]:
import numpy as np
image_paths = np.asarray(all_image_paths)
sorted_image_paths = np.sort(image_paths)
print(sorted_image_paths)

annotation_paths = np.asarray(all_annotation_paths)
sorted_annotation_paths = np.sort(annotation_paths)
print(sorted_annotation_paths)

# Cropping the Images

In [ ]:
def findIndexofElement(value, array):
  for i in range(len(array)):
    if array[i][array[i].rfind('/'):] == value:
      return i
      break

In [ ]:
os.mkdir('/content/allCroppedImages/')
os.mkdir('/content/allCroppedImages/stop/')
os.mkdir('/content/allCroppedImages/warning/')
os.mkdir('/content/allCroppedImages/go/')
os.mkdir('/content/allCroppedImages/warningLeft/')
os.mkdir('/content/allCroppedImages/goLeft/')
os.mkdir('/content/allCroppedImages/stopLeft/')
os.mkdir('/content/allCroppedImages/goForward/')

In [ ]:
from google.colab.patches import cv2_imshow
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.utils import array_to_img
from tensorflow.keras.utils import load_img
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from PIL import Image
import os

def cropAllImages(path):
  df = pd.read_csv(path, sep=';')
  filenames = df['Filename']
  leftX = np.asarray(df['Upper left corner X'])
  rightX = np.asarray(df['Lower right corner X'])
  leftY = np.asarray(df['Upper left corner Y'])
  rightY = np.asarray(df['Lower right corner Y'])
  tag = np.asarray(df['Annotation tag'])

  # loc_index is the location of the image path in all sorted paths
  for i in range(len(filenames)):
    findIndexofElement(filenames[i][filenames[i].rfind('/'):], sorted_image_paths)
    img = img_to_array(load_img(sorted_image_paths[findIndexofElement(filenames[i][filenames[i].rfind('/'):], sorted_image_paths)]))
    crop_img = array_to_img(img[leftY[i]:rightY[i], leftX[i]:rightX[i]])

    # inputting them in folder
    crop_img.save('/content/allCroppedImages/' + tag[i] + filenames[i][filenames[i].rfind('/'):])

for path in all_annotation_paths:
  cropAllImages(path)

TypeError: ignored

## Randomly Assigning Files

- after putting into sub-folders
- for each subfolder:
  - put all names in a list
  - shuffle
  - get all three indices
  - put into train, test, val

In [ ]:
os.mkdir('/content/train/')
os.mkdir('/content/test/')
os.mkdir('/content/val/')

folderList = ['train', 'test', 'val']
for name in folderList:
  os.mkdir('/content/' + name + '/stop/')
  os.mkdir('/content/' + name + '/go/')
  os.mkdir('/content/' + name + '/warning/')
  os.mkdir('/content/' + name + '/warningLeft/')
  os.mkdir('/content/' + name + '/goLeft/')
  os.mkdir('/content/' + name + '/stopLeft/')
  os.mkdir('/content/' + name + '/goForward/')

In [ ]:
def listPaths(path):
  pathList = []
  for folder in os.listdir(path):
    if folder == '.ipynb_checkpoints':
      continue
    pathList.append(path + folder + '/')
  return pathList

print(listPaths('/content/allCroppedImages/'))

['/content/allCroppedImages/warningLeft/', '/content/allCroppedImages/stopLeft/', '/content/allCroppedImages/warning/', '/content/allCroppedImages/goForward/', '/content/allCroppedImages/go/', '/content/allCroppedImages/goLeft/', '/content/allCroppedImages/stop/']


In [ ]:
import random
import shutil

def shuffleSelection(path):
  allFolders = listPaths(path)
  for folder in allFolders:
    df = pd.DataFrame(listPaths(folder))
    trainPaths, testPaths, valPaths = np.split(df, [int(.8 * len(df)), int(.9 * len(df))])
    moveToFolder(trainPaths, testPaths, valPaths)

def moveToFolder(trainPaths, testPaths, valPaths):
  finalTrainPathList = np.asarray(trainPaths[0])
  finalTestPathList = np.asarray(testPaths[0])
  finalValPathList = np.asarray(valPaths[0])
  for path in finalTrainPathList:
    shutil.move(path[:-1], '/content/train' + path[25:-1])
  for path in finalTestPathList:
    shutil.move(path[:-1], '/content/test' + path[25:-1])
  for path in finalValPathList:
    shutil.move(path[:-1], '/content/val' + path[25:-1])
  

In [ ]:
shuffleSelection('/content/allCroppedImages/')

KeyError: ignored

In [ ]:
# import os, shutil
# folder = '/content/allCroppedImages/'
# for filename in os.listdir(folder):
#     file_path = os.path.join(folder, filename)
#     try:
#         if os.path.isfile(file_path) or os.path.islink(file_path):
#             os.unlink(file_path)
#         elif os.path.isdir(file_path):
#             shutil.rmtree(file_path)
#     except Exception as e:
#         print('Failed to delete %s. Reason: %s' % (file_path, e))

# Training Dataset

## Agenda for 10/18

- sort out issue with random shuffle function (ask about cropping time)
- find way to convert images to dataset
  - ask why we need singular class folders

Goals for the next two weeks:
- run experiments (and caputre results)
- finish research paper